In [1]:
# LT 15/10/2022
# for the 10 genes testing panel
# extract mutational model expected values from RMC project

import hail as hl
import json

hl.init(default_reference='GRCh38',
        spark_conf = {
            'spark.hadoop.fs.gs.requester.pays.mode': 'CUSTOM',
            'spark.hadoop.fs.gs.requester.pays.buckets': 'regional_missense_constraint',
            'spark.hadoop.fs.gs.requester.pays.project.id': 'constraint-232598',
        })

2022-11-05 05:14:17 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-11-05 05:14:19 WARN  Utils:69 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2022-11-05 05:14:19 WARN  Utils:69 - Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Running on Apache Spark version 3.1.3
SparkUI available at http://0c2923260f02:4042
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.98-f8833c1ae16b
LOGGING: writing to /home/jupyter/structural-constraint/gnomad/prototype/hail-20221105-0514-0.2.98-f8833c1ae16b.log


In [12]:
# RMC results table
rmc_ht = hl.read_table('gs://regional_missense_constraint/model/2.1.1/1/context_obs_exp_annot.ht')

In [21]:
# json file with list of transcripts
transcripts_file = open("transcripts.json", "r")
TRANSCRIPTS = json.load(transcripts_file)

# iterate on transcripts
for current_transcript in TRANSCRIPTS:
    
    # filter to current canonical transcript
    ht = rmc_ht.filter(rmc_ht.transcript == current_transcript)
    
    # annotate with the cumulative exp of previous row
    ht = ht.annotate(prev_row_cumexp=hl.scan.take(ht.cumulative_exp,1,ordering=-ht.cumulative_exp).first())

    # calculate locus-specific expected by taking the difference
    ht = ht.annotate(exp = ht.cumulative_exp - hl.coalesce(ht.prev_row_cumexp,0))

    # keep what we need and rename observed to obs
    ht = ht.select(ht.exp, obs = ht.observed)
    
    # export
    ht.export("{0}_bp_oe.tsv".format(current_transcript))

2022-11-05 06:17:36 Hail: INFO: merging 16402 files totalling 37.2K...) / 16401]
2022-11-05 06:17:38 Hail: INFO: while writing:
    ENST00000366684_bp_oe.tsv
  merge time: 2.263s
2022-11-05 06:24:27 Hail: INFO: merging 16402 files totalling 42.0K...) / 16401]
2022-11-05 06:24:29 Hail: INFO: while writing:
    ENST00000371953_bp_oe.tsv
  merge time: 2.249s
2022-11-05 06:31:27 Hail: INFO: merging 16402 files totalling 262.1K... / 16401]
2022-11-05 06:31:29 Hail: INFO: while writing:
    ENST00000298910_bp_oe.tsv
  merge time: 2.248s
2022-11-05 06:39:00 Hail: INFO: merging 16402 files totalling 12.8K...) / 16401]
2022-11-05 06:39:03 Hail: INFO: while writing:
    ENST00000320868_bp_oe.tsv
  merge time: 2.702s
2022-11-05 06:46:23 Hail: INFO: merging 16402 files totalling 84.7K...) / 16401]
2022-11-05 06:46:25 Hail: INFO: while writing:
    ENST00000359125_bp_oe.tsv
  merge time: 2.241s
2022-11-05 06:53:12 Hail: INFO: merging 16402 files totalling 25...+ 3) / 16401]
2022-11-05 06:53:14 Hail